In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from scipy.stats import pearsonr
import math
from scipy import spatial
from pmdarima import auto_arima
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from math import sqrt
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [14]:
#confirmed = pd.read_csv('time_series_covid19_confirmed_global.csv')
# confirmed = pd.read_csv('time_series_covid19_deaths_global.csv')
# recovered = pd.read_csv('time_series_covid19_recovered_global.csv')

# latest
confirmed = pd.read_csv('time_series_covid19_recovered_global.csv')
#confirmed= pd.read_csv('deaths_global_latest.csv')

# Util

In [15]:
def scorer(y_pred, y_test):
    return np.sum([(np.log10(x) - np.log10(y))**2 for x, y in zip(y_pred, y_test) if x>1 and y>100])

In [16]:
def evaluate(sorted_combined):
    n = len(sorted_combined[0][2])
    scores = []
    for day in range(n):
        y_real = []
        y_pred = []
        for a in sorted_combined:
            
            y_real = y_real + [a[2]['cases'].values[day]]
            y_pred = y_pred + [a[3]['Prediction'].values[day]]

        y_pred = [int(np.ceil(x)) for x in y_pred]
        sc = scorer(y_pred, y_real)
        print(day, sc)
        scores.append(sc)
    return scores, y_pred, y_real

In [17]:
def evaluate_ensemble(sorted_combined, sorted_combined1):
    n = len(sorted_combined[0][2])
    scores = []
    for day in range(n):
        y_real = []
        y_pred = []
        y_pred1 = []
        for i in range(len(sorted_combined)):
            a = sorted_combined[i]
            b = sorted_combined1[i]
            y_real = y_real + [a[2]['cases'].values[day]]
            y_pred = y_pred + [a[3]['Prediction'].values[day]]
            y_pred1 = y_pred1 + [b[3]['Prediction'].values[day]]

        y_pred_ensembled = [int(np.ceil((x+x1)/2.0) ) for x,x1 in zip(y_pred, y_pred1)]
        sc = scorer(y_pred_ensembled, y_real)
        print(day, sc)
        scores.append(sc)
    return scores, y_pred_ensembled,y_pred, y_pred1,  y_real

# Auto Arima

In [18]:
a=10
columns = confirmed.columns[4:]
df = pd.DataFrame({'cases':confirmed.iloc[a].values[4:]})
df.index = pd.to_datetime(columns)
train = df
valid = df[-8:]
print(train['cases'][-2])

9


In [19]:
def my_arima_model(index, train, valid, do_plot=0):
    if train['cases'][-1]==train['cases'][-2] and train['cases'][-2]==train['cases'][-3]:
        forecast = [train['cases'][-1]]*(len(valid))
        print('......', forecast)
    else:
        model = auto_arima(train, trace=True, error_action='ignore', suppress_warnings=True, information_criterion='bic')
        model.fit(train)
        forecast = model.predict(n_periods=len(valid))
        print('---------', forecast)

    forecast = pd.DataFrame(forecast,index = valid.index,columns=['Prediction'])
    rms = sqrt(mean_squared_error(valid,forecast))
    print(index, rms)
    #plot the predictions for validation set
    if do_plot==1:
        plt.plot(train, label='Train')
        plt.plot(valid, label='Valid')
        plt.plot(forecast, label='Prediction')
        plt.legend()
        plt.show()
    return rms, forecast

In [20]:
combined = []
columns = confirmed.columns[4:]
for a in range(len(confirmed)):
    df = pd.DataFrame({'cases':confirmed.iloc[a].values[4:]})
    df.index = pd.to_datetime(columns)
    train = df
    valid = df[-8:]
    rms, forecast = my_arima_model(a, train, valid)
    current = [rms, a, valid, forecast, train]
    combined.append(current)
    

Performing stepwise search to minimize bic
Fit ARIMA: (2, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=551.770, BIC=566.835, Time=0.118 seconds
Fit ARIMA: (0, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=581.158, BIC=586.179, Time=0.009 seconds
Fit ARIMA: (1, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=558.118, BIC=565.650, Time=0.033 seconds
Fit ARIMA: (0, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=547.545, BIC=555.077, Time=0.048 seconds
Fit ARIMA: (0, 2, 0)x(0, 0, 0, 0) (constant=False); AIC=579.213, BIC=581.724, Time=0.013 seconds
Fit ARIMA: (1, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=549.177, BIC=559.220, Time=0.075 seconds
Fit ARIMA: (0, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=549.280, BIC=559.323, Time=0.051 seconds
Fit ARIMA: (1, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=551.542, BIC=564.096, Time=0.111 seconds
Total fit time: 0.466 seconds
--------- [193.40194573 207.96542947 222.6904512  237.57701094 252.62510868
 267.83474442 283.20591817]
0 99.14734670521909
Performing stepwise search to mini

In [10]:
score, y_pred, y_real = evaluate(combined)
print("Score ",score)
print("Prediction, Actual")
arima_pred_df = pd.DataFrame({"Actual": y_real, "Prediction":y_pred})
#arima_pred_df.to_csv("21_april_deaths_arima_pred_df.csv", index=False)
#arima_pred_df.head()

0 7.276201734390241
1 7.215862326474318
2 6.937898943398325
3 7.234268652664489
4 7.371984195689572
5 6.789852801618909
6 5.548243653445059
7 4.603850781204079
Score  [7.276201734390241, 7.215862326474318, 6.937898943398325, 7.234268652664489, 7.371984195689572, 6.789852801618909, 5.548243653445059, 4.603850781204079]
Prediction, Actual


# Holt winter

In [11]:
def my_arima_model(index, train, valid, do_plot=0):
    if train['cases'][-1]==train['cases'][-2] and train['cases'][-2]==train['cases'][-3]:
        pred1 = [train['cases'][-1]]*(len(valid))
        pred2 = [train['cases'][-1]]*(len(valid))
        print('......', forecast)
    else:
        model1 = ExponentialSmoothing(np.asarray(train['cases']), trend="add", seasonal="add", seasonal_periods=7)
        model2 = ExponentialSmoothing(np.asarray(train['cases']), trend="add", seasonal="add", seasonal_periods=7, damped=True)
        fit1 = model1.fit()
        pred1 = fit1.forecast(len(valid))
        fit2 = model2.fit()
        pred2 = fit2.forecast(len(valid))
        
    forecast1 = pd.DataFrame(pred1,index = valid.index,columns=['Prediction'])
    rms1 = sqrt(mean_squared_error(valid,forecast1))
    forecast2 = pd.DataFrame(pred2,index = valid.index,columns=['Prediction'])
    rms2 = sqrt(mean_squared_error(valid,forecast2))
    
    print(index, rms1, rms2)
    
    #plot the predictions for validation set
    if do_plot==1:
        plt.plot(train, label='Train')
        plt.plot(valid, label='Valid')
        plt.plot(forecast, label='Prediction')
        plt.legend()
        plt.show()
    return rms1, forecast1, rms2, forecast2

In [12]:
combined1 = []
combined2 = []
columns = confirmed.columns[4:]
for a in range(len(confirmed)):
    df = pd.DataFrame({'cases':confirmed.iloc[a].values[4:], 'date':pd.to_datetime(columns)}).set_index('date')
    #df.index = pd.to_datetime(columns)
    train = df
    valid = df[-7:]
    rms1, forecast1, rms2, forecast2 = my_arima_model(a, train, valid)
    current1 = [rms1, a, valid, forecast1, train]
    combined1.append(current1)
    current2 = [rms2, a, valid, forecast2, train]
    combined2.append(current2)
    

0 113.30149170163435 113.26617505466714
1 137.29766083214506 137.29916551925763
2 682.6971854248904 682.6823489018684
3 202.58200400135277 204.72394070993838
......             Prediction
2020-04-16           0
2020-04-17           0
2020-04-18           0
2020-04-19           0
2020-04-20           0
2020-04-21           0
2020-04-22           0
2020-04-23           0
4 0.5 0.5
5 9.154130997341802 5.635034043826012
6 344.41347064261817 344.4237666464768
7 230.9872214392695 213.46855108903438
......             Prediction
2020-04-16           0
2020-04-17           0
2020-04-18           0
2020-04-19           0
2020-04-20           0
2020-04-21           0
2020-04-22           0
2020-04-23           0
8 3.2596012026013246 3.2596012026013246
......             Prediction
2020-04-16           0
2020-04-17           0
2020-04-18           0
2020-04-19           0
2020-04-20           0
2020-04-21           0
2020-04-22           0
2020-04-23           0
9 486.1359120657514 486.1359120657

KeyboardInterrupt: 

In [ ]:
score, y_pred, y_real = evaluate(combined1)
print("Score ",score)
print("Prediction, Actual")
arima_pred_df = pd.DataFrame({"Actual": y_real, "Prediction":y_pred})
#arima_pred_df.to_csv("22_april_cases_holt_pred_df.csv", index=False)
#arima_pred_df.head()

In [ ]:
score, y_pred_7thday, y_pred, y_pred1, y_real  = evaluate_ensemble(combined, combined1)

# Ensembled

In [21]:
ensembled_Results = pd.DataFrame({'7th_yReal': y_real, '7th_yArima': y_pred, '7th_yHolt': y_pred, '7th_yEnsembled': y_pred})
ensembled_Results.to_csv("latest_ensembled_recovered.csv", index=False)